In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.sampler import WeightedRandomSampler

from pytorch_lightning import LightningDataModule
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.strategies import ParallelStrategy
from pytorch_lightning.utilities.cli import LightningCLI
from pytorch_lightning import Trainer
# from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.plugins import DDPPlugin
from torchmetrics import Accuracy, F1Score, Specificity

import json
import matplotlib.image as image 
import albumentations as A
import albumentations.pytorch
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../')

from utils.dataset import *



/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../dataset/df.csv')

In [3]:
df.head()

,image_id,image_dir,category,genus,institutions
0,00000__001,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
1,00000__002,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
2,00000__003,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
3,00000__004,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
4,00000__005,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0


In [4]:
df.shape

(839772, 5)

In [5]:
print(len(df.category.unique()))
print(len(df.genus.unique()))
print(len(df.institutions.unique()))

15501
2564
60


In [6]:
df.reset_index(inplace=True, drop=True)
df.head()

,image_id,image_dir,category,genus,institutions
0,00000__001,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
1,00000__002,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
2,00000__003,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
3,00000__004,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
4,00000__005,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0


In [9]:
min(df.category.unique())

0

In [5]:
class HerbClsModel(LightningModule) :
    def __init__(
        self,
        arch: str = 'resnet18',
        pretrained: bool = False,
        lr: float = 0.9,
        momentum: float = 0.9,
        weight_decay: float = 1e-4,
        batch_size: int =256,
        workers: int = 4,
        num_classes: int = 5,
        from_contra : str = './saved_models/contra/',
        # is_contra: bool = False,
    ):
        
        super().__init__()
        self.arch = arch
        self.pretrained = pretrained
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.batch_size = batch_size
        self.workers = workers
        self.num_classes = num_classes
        self.from_contra = from_contra
        
        if self.arch not in models.__dict__.keys() : 
            # self.model = custom_models.__dict__[self.arch](pretrained=False, img_size=args.img_size)
            self.model = custom_models.__dict__[self.arch](pretrained=False)
        else :
            print('only resnet is supported') 
            self.model = models.__dict__[self.arch](pretrained=self.pretrained) 
        
        shape = self.model.fc.weight.shape
        self.model.fc = nn.Linear(shape[1], self.num_classes)
        self.criterion = nn.CrossEntropyLoss()
            
        print("=> creating model '{}'".format(self.arch))
        self.train_acc1 = Accuracy(top_k=1)
        self.eval_acc1 = Accuracy(top_k=1)
        self.f1 = F1Score(average='macro', num_classes=self.num_classes)
        self.specificity = Specificity(average='macro', num_classes=self.num_classes)
        
        self.save_hyperparameters()
        
    def forward(self, x) :
        return self.model(x)

    def training_step(self, batch, batch_idx) :
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        correct=outputs.argmax(dim=1).eq(targets).sum().item()
        total=len(targets)
        
        #update metric
        self.log('train_loss', loss)
        self.train_acc1(outputs, targets)
        self.log('train_acc', self.train_acc1, prog_bar=True)
        
        #for tensorboard
        logs={"train_loss": loss}
        batch_dictionary={
            'loss':loss,
            'log':logs,
            # info to be used at epoch end
            "correct": correct,
            "total": total
        }        
        
        return batch_dictionary
    
    def training_epoch_end(self, outputs):
        # if(self.current_epoch==1):
        #     sampleImg=torch.rand((1,3,IMAGE_SIZE,IMAGE_SIZE))
        #     self.logger.experiment.add_graph(self.model(), sampleImg)

        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        
        # logging histograms
        # self.custom_histogram_adder()
        
        # calculating correect and total predictions
        correct=sum([x["correct"] for  x in outputs])
        total=sum([x["total"] for  x in outputs])
        
        # creating log dictionary
        # self.logger.experiment.add_scalar('loss/train', avg_loss, self.current_epoch)
        # self.logger.experiment.add_scalar('acc/train', correct/total, self.current_epoch)
        tensorboard_logs = {'loss': avg_loss, "Accuracy": correct/total}
        
        epoch_dictionary={
            # required
            'loss': avg_loss,
            # for logging purposes
            'log': tensorboard_logs
        }
 
        # wandb expect None
        # return epoch_dictionary  

    def custom_histogram_adder(self):
        for name, params in self.named_parameters():
            self.logger.experiment.add_histogram(name, params, self.current_epoch)
    
    def eval_step(self, batch, batch_idx, prefix: str) :
        images, targets = batch
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        
        self.log(f'{prefix}_loss', loss)
        self.eval_acc1(outputs, targets)
        self.log(f'{prefix}_acc1', self.eval_acc1, prog_bar=True)
        self.f1(outputs, targets)
        self.log(f'{prefix}_f1_score', self.f1, prog_bar=True)
        self.specificity(outputs, targets)
        self.log(f'{prefix}_specificity', self.specificity, prog_bar=True)    
        
        if prefix == 'val' :
            correct=outputs.argmax(dim=1).eq(targets).sum().item()
            total=len(targets) 
            
            #for tensorboard
            logs={"val_loss": loss}
            batch_dictionary={
                'loss':loss,
                'log':logs,
                # info to be used at epoch end
                "correct": correct,
                "total": total
            }  
            
            return batch_dictionary

        return loss            
        
    def validation_step(self, batch, batch_idx) :
        return self.eval_step(batch, batch_idx, 'val')
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        
        # calculating correect and total predictions
        correct=sum([x["correct"] for  x in outputs])
        total=sum([x["total"] for  x in outputs])
        
        tensorboard_logs = {'loss': avg_loss, "Accuracy": correct/total}
        
        epoch_dictionary={
            # required
            'loss': avg_loss,
            # for logging purposes
            'log': tensorboard_logs
        }
        
        # wandb expect None
        # return epoch_dictionary    

    def test_step(self, batch, batch_idx) :
        return self.eval_step(batch, batch_idx, 'test')
    
    def configure_optimizers(self) :
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, self.parameters()), 
                              lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
        
        scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch : 0.1 **(epoch //30))
        return [optimizer], [scheduler]
    
#     load contra checkpoint in fine tuning
    def load_contra_checkpoint(self, path):
        state_dict = torch.load(path)['state_dict']
        model_state_dict = self.state_dict()
        for k in state_dict:
            if k in model_state_dict:
                if state_dict[k].shape != model_state_dict[k].shape:
                    print(f"Skip loading parameter: {k}, "
                                f"required shape: {model_state_dict[k].shape}, "
                                f"loaded shape: {state_dict[k].shape}")
                    state_dict[k] = model_state_dict[k]

            else:
                print(f"Dropping parameter {k}")

        self.load_state_dict(state_dict)
        
#     model freezing when fine tuning( linear evaluation protocol)
    def model_freeze(self) :
        for param in self.parameters( ) :
            param.requires_grad = False
        
        self.model.fc.weight.requires_grad = True
        self.model.fc.bias.requires_grad = True

In [6]:
df.head()

,image_id,image_dir,category,genus,institutions
0,00000__001,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
1,00000__002,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
2,00000__003,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
3,00000__004,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
4,00000__005,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0


In [7]:
df.loc[10, 'category']

0

In [8]:
df.reset_index(inplace=True)
df.head()

,index,image_id,image_dir,category,genus,institutions
0,0,00000__001,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
1,1,00000__002,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
2,2,00000__003,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
3,3,00000__004,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0
4,4,00000__005,../dataset/herbarium_2022/train_images/000/00/...,0,Abies,0


In [9]:
batch_size = 32
dm = HerbDataModule(df, batch_size, transform=train_transforms)


(839772, 6)


/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:95: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


In [ ]:
num_classes = len(df.category.unique())
model = HerbClsModel(pretrained=True, num_classes=num_classes)

devices = torch.cuda.device_count()
trainer = Trainer(
    max_epochs=3,
    progress_bar_refresh_rate=20,
    # gpus=devices,
    gpus=1,
    # strategy = "ddp",
)
trainer.fit(model, dm)

only resnet is supported


/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


=> creating model 'resnet18'


/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:126: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


(629829, 7)
(209943, 7)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | ResNet           | 19.1 M
1 | criterion   | CrossEntropyLoss | 0     
2 | train_acc1  | Accuracy         | 0     
3 | eval_acc1   | Accuracy         | 0     
4 | f1          | F1Score          | 0     
5 | specificity | Specificity      | 0     
-------------------------------------------------
19.1 M    Trainable params
0         Non-trainable params
19.1 M    Total params
76.514    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 40/26244 [00:45<8:21:48,  1.15s/it, loss=9.7, v_num=5, train_acc=0.000] 

In [ ]:
df.head()

In [ ]:
len(df.category.unique())